$\def \dot #1#2{\left\langle #1, #2 \right\rangle}$
$\def \adot #1#2{\left\langle #1, #2 \right\rangle}$
$\def \cA {\mathcal{A}}$
$\def \cD {\mathcal{D}}$
$\def \cM {\mathcal{M}}$
$\def \cN {\mathcal{N}}$
$\def \cW {\mathcal{W}}$
$\def \bc {\mathbf{c}}$
$\def \bu {\mathbf{u}}$
$\def \bv {\mathbf{v}}$
$\def \bw {\mathbf{w}}$
$\def \bG {\mathbf{G}}$
$\def \bC {\mathbf{C}}$
$\def \bD {\mathbf{D}}$
$\def \bI {\mathbf{I}}$
$\def \bP {\mathbf{P}}$
$\def \bQ {\mathbf{Q}}$
$\def \bR {\mathbf{R}}$
$\def \bS {\mathbf{S}}$
$\def \bT {\mathbf{T}}$
$\def \bU {\mathbf{U}}$
$\def \bV {\mathbf{V}}$
$\def \bW {\mathbf{W}}$
$\def \bPhi {\mathbf{\Phi}}$
$\def \bPsi {\mathbf{\Psi}}$
$\def \bGamma {\mathbf{\Gamma}}$
$\def \bSigma {\mathbf{\Sigma}}$
$\def \bTheta {\mathbf{\Theta}}$
$\def \bOmega {\mathbf{\Omega}}$
$\def \bbE {\mathbb{E}}$
$\def \bbP {\mathbb{P}}$
$\def \bbR {\mathbb{R}}$
$\def \bbN {\mathbb{N}}$

# Now let us consider the PDE problem 

We compute $N$ snapshots, we truncate the PCA at some $n$, we calculate $\bC$ and the associated PCA decomposition

In [1]:
import numpy as np
import scipy as sp
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import pdb

import sys
sys.path.append("../../")
import pyApproxTools as pat
importlib.reload(pat)

%matplotlib inline

# Constants taken throughout this workbook
fem_div = 7
a_bar = 1.0
c = 0.9
field_div = 2
side_n = 2**field_div

### Generate our dictionary of points. Effectively this is what our PCA will be computed from

In [2]:
N = int(1e3)
np.random.seed(1)

dict_basis, dict_fields = pat.make_pw_reduced_basis(N, field_div, fem_div, a_bar=a_bar, c=c, f=1.0, verbose=False)
dictionary = dict_basis.vecs

In [3]:
# local_width is the width of the measurement squares in terms of FEM mesh squares
width_div = 1
local_width = 2**width_div
spacing_div = 4

Wm_reg, Wloc_reg = pat.make_local_avg_grid_basis(width_div, spacing_div, fem_div, return_map=True)
Wm_reg = Wm_reg.orthonormalise()

m = Wm_reg.n

Wm_rand, Wloc = pat.make_pw_local_avg_random_basis(m=m, div=fem_div, width=local_width, return_map=True)
Wm_rand = Wm_rand.orthonormalise()

Once again we define the operators $U :\bbR^N\to V$ as $U x = \sum_{i=1}^N x_i u_i$ and evidently $U^* v = (\langle u_i, v \rangle_V )_{i=1}^N$. We have $C = U U^*$ and $G = U^* U$. It is straightforward to see that $G$ is represented by a matrix $(\bG)_{i,j} = \langle u_i, u_j \rangle$.

Now - this is where things are a little sensitive. Say that $V_n = \mathrm{span}(u_1,\ldots,u_N)$, clearly $n\le N$. Now, $v = P_{V_n} v + P_{V_n^\perp}v$, and obviously $U^* P_{V_n^\perp} = 0$


$$
\bC =  \bU \bU^T  = \bPhi \bSigma \bSigma^T \bPhi^T \quad\text{and}\quad \bG = \bU^T \bU = \bTheta \bSigma^T \bSigma \bTheta^T
$$

Note that $\bSigma \bSigma^T$ is a diagonal $K\times K$ matrix, while $\bSigma^T \bSigma$ is $N\times N$, they both are diagonal with $\sigma_i^2$ along the diagonal.

Now, we have that $\bPhi = \bU \bTheta \bSigma^{-1}$. I'm reasonably sure this all applies if we consider a more general $V$, with of course the addition of an operator $E : v \to \bbR^K$ that maps from a canonical ortho basis to the coordinates. But this doesn't complicate things too much.

First let us test this all in $\bbR^K$ for some moderate $K$.

In [ ]:
u0 = dict_basis.reconstruct(np.ones(N) / N)

import copy

cent_vecs = copy.deepcopy(dict_basis.vecs)
for i in range(len(cent_vecs)):
    cent_vecs[i] = cent_vecs[i] - u0

dict_basis = pat.PWBasis(cent_vecs)
dict_basis.make_grammian()

lam, V = np.linalg.eigh(dict_basis.G)
PCA_vecs = [u0]
for i, v in enumerate(np.flip(V.T, axis=0)[:m]):
    vec = dict_basis_small_cent.reconstruct(v)
    PCA_vecs.append(vec / vec.norm())

Vn_PCA = pat.PWBasis(PCA_vecs)

